# 대구시 중구 10년치 매매거래와 거래된 아파트의 세부정보를 얻기위한 코드

데이터는 https://www.data.go.kr 여기에서 API를 사용해서 받았습니다

필요한 라이브러리 준비

In [1]:
import urllib
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
from dateutil.rrule import rrule, MONTHLY
import csv
import sys
import numpy as np
import json

# Get Data

In [1]:
'''
    Get data using API and save to csv file.
'''

deal_ymd = []
data_column = ['거래금액', '건축년도', '년', '법정동', '아파트', '월', '일', '전용면적', '지번', '지역코드', '층']

# 10년치 데이터를 얻기위해 년도와 월을 만들기위한 함수
def month_iter(start_year, start_month, end_year, end_month):
    start = datetime(start_year, start_month, 1)
    end = datetime(end_year, end_month, 1)

    for d in rrule(MONTHLY, dtstart=start, until=end):
        deal_ymd.append(str(d.year) + str(month_printer(d)))

# 월 변환 함수
def month_printer(dateObj):
    if dateObj.month < 10:
        return '0' + str(dateObj.month)
    else:
        return dateObj.month

# 2007년 8월 ~ 2017년 8월 까지의 데이터
month_iter(2007, 8, 2017, 8)

# 받은 데이터를 csv파일로 저장
with open('jib.csv', "w", newline='') as csv_file:
        writer = csv.writer(csv_file, delimiter=',')
        # write header
        writer.writerow(data_column)
        
        # 키 값은 https://www.data.go.kr 여기에 들어가서 자료신청을 하면 개인별 고유키값을 줍니다.
        # 키 값 2007.8 ~ 2017.8
        for d in deal_ymd:
            key = '&serviceKey=뭐라뭐라~~ 자기 키값넣으시면됩니다.'
            URL = 'http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTrade?LAWD_CD=27110&DEAL_YMD=' + d + key
            res = urllib.request.urlopen(URL)
            # xml포맷을 파싱
            soup = BeautifulSoup(res, "xml")
            # 페이지에 있는 모든 아이템을 찾는다.
            items = soup.find_all("item")

            # 아이템들을 순회하면서
            # 그 안에 태그안 데이터들을 리스트에 붙여넣습니다.
            # 그렇게 하나의 데이터가 끝나면 csv에 쓰고 다음 데이터로
            for item in items:
                deal = []
                deal.append(item.find("거래금액").text)
                deal.append(item.find("건축년도").text)
                deal.append(item.find("년").text)
                deal.append(item.find("법정동").text)
                deal.append(item.find("아파트").text)
                deal.append(item.find("월").text)
                deal.append(item.find("일").text)
                deal.append(item.find("전용면적").text)
                deal.append(item.find("지번").text)
                deal.append(item.find("지역코드").text)
                deal.append(item.find("층").text)
                writer.writerow(deal)


In [2]:
# 저장한 데이터를 테이블로 읽어옵니다.
df = pd.read_csv('jib.csv')
df.head(10)

,거래금액,건축년도,년,법정동,아파트,월,일,전용면적,지번,지역코드,층
0,"14,500",2007,2007,동인동1가,삼승슈퍼타워,8,1~10,83.50,362,27110,7
1,"15,300",2007,2007,동인동1가,삼승슈퍼타워,8,21~31,83.50,362,27110,14
2,"16,920",2007,2007,동인동1가,삼승슈퍼타워,8,21~31,83.50,362,27110,11
3,"16,000",2006,2007,봉산동,봉산뜨란채,8,1~10,75.95,110,27110,3
4,"9,200",1975,2007,수동,희도맨션,8,1~10,101.26,101,27110,8
5,"5,800",1985,2007,태평로1가,태평라이프,8,1~10,54.60,1-187,27110,8
6,"5,500",1985,2007,태평로1가,태평라이프,8,1~10,54.60,1-187,27110,6
7,"5,100",1976,2007,도원동,도원,8,1~10,38.91,3-5,27110,2
8,"12,500",2007,2007,동산동,석우리치타운,8,1~10,81.90,5-2,27110,4
9,"12,500",2007,2007,동산동,석우리치타운,8,11~20,81.90,5-2,27110,3


#### 거래금액은 기본단위가 만원

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6926 entries, 0 to 6925
Data columns (total 11 columns):
거래금액    6926 non-null object
건축년도    6926 non-null int64
년       6926 non-null int64
법정동     6926 non-null object
아파트     6926 non-null object
월       6926 non-null int64
일       6926 non-null object
전용면적    6926 non-null float64
지번      6926 non-null object
지역코드    6926 non-null int64
층       6926 non-null int64
dtypes: float64(1), int64(5), object(5)
memory usage: 595.3+ KB


# 데이터 만지기

## 데이터에 대해 3가지를 따져봐야합니다.
1. null value 가 데이터 안에 있는지
2. 각 데이터들에(문자열) 공백이 있는지
3. 데이터 형이 적절한지

In [4]:
# 데이터 안에 공백이 있는지 쉽게 알기위해 numpy 리스트로 읽고 랜덤으로 아무거나 하나 봅니다.

# 파일읽기
with open('jib.csv', newline='') as f:
    reader = csv.reader(f, delimiter=',')
    deal_data = [row for row in reader]
# 리스트를 numpy array 로 변환.
deal_data = np.array(deal_data)

# 아무거나 하나 선택해서 보기
print(deal_data[124])

['     9,250' '1992' '2007' ' 남산동' '송림맨션' '11' '21~30' '59.99' '2434'
 '27110' '16']


거래금액과 법정동에 공백이 있네요. 고쳐봅시다

그리고 거래금액의 경우 우리가 예측하려는 타겟변수입니다. 꼭 int나 float 숫자형으로 바꿔줘야합니다.

In [5]:
# null 이 있는지 확인
df.isnull().sum()

거래금액    0
건축년도    0
년       0
법정동     0
아파트     0
월       0
일       0
전용면적    0
지번      0
지역코드    0
층       0
dtype: int64

In [6]:
# 거래금액 데이터를 문자열에서 숫자로 바꿔줘야 합니다.
# 일단 strip()으로 공백을 제거후,
# replace()로 데이터 내의 콤마를 없애줍니다.
# 그리고 자료형을 숫자형으로 변환

df['거래금액'] = df['거래금액'].apply(lambda x: int(x.strip().replace(',', '')))
df['거래금액'][:5]

0    14500
1    15300
2    16920
3    16000
4     9200
Name: 거래금액, dtype: int64

잘 됬네요.

In [7]:
# 법정동데이터에 공백을 없애줍니다.
df['법정동'] = df['법정동'].apply(lambda x: x.strip())

In [8]:
# set을 이용해 법정동안에 있는 값들을 얻습니다. 
dong_list = set(df['법정동'])

# set을 리스트로 변환
dong_list = list(dong_list)
dong_list

['동산동',
 '수동',
 '수창동',
 '장관동',
 '태평로2가',
 '계산동2가',
 '교동',
 '삼덕동2가',
 '동인동3가',
 '도원동',
 '남산동',
 '동인동4가',
 '시장북로',
 '삼덕동3가',
 '대봉동',
 '달성동',
 '태평로1가',
 '서성로1가',
 '봉산동',
 '대신동',
 '태평로3가',
 '동인동1가']

끝!

In [9]:
# 아파트코드를 받아오기 위해서는 loadcode;법정동코드를 알아야한다.
# 직접 code.go.kr에서 중구 전체의 동을 엑셀파일로 받아옴
# 받은 파일을 읽어와서 실거래 데이터에 포함된 동들을 찾아 코드 저장

dong_code_df = pd.read_excel('junggu_dongcode.xls')
dong_code_df.head(10)

,법정동코드,법정동명
0,2711000000,대구광역시 중구
1,2711010100,대구광역시 중구 동인동1가
2,2711010200,대구광역시 중구 동인동2가
3,2711010300,대구광역시 중구 동인동3가
4,2711010400,대구광역시 중구 동인동4가
5,2711010500,대구광역시 중구 삼덕동1가
6,2711010600,대구광역시 중구 삼덕동2가
7,2711010700,대구광역시 중구 삼덕동3가
8,2711010800,대구광역시 중구 봉산동
9,2711010900,대구광역시 중구 장관동


In [10]:
# dong_code_df에서 첫번째 행은 동에대한 정보가 없으므로 삭제합니다.
dong_code_df.drop(dong_code_df.index[0], inplace=True)

In [11]:
# 우리가 원하는 것은 법정동이므로
# 법정동명을 공백으로 나누면 ex. 대구광역시 중구 동인동1가 --> [대구광역시, 중구, 동인동1가] 이렇게 3개로 나뉩니다.
# 이중 마지막것이 우리가 원하는 동이름이므로 3번째 것으로 다시 저장합니다.

dong_code_df['법정동명'] = dong_code_df['법정동명'].apply(lambda x: x.split()[2])

In [12]:
dong_code_df

,법정동코드,법정동명
1,2711010100,동인동1가
2,2711010200,동인동2가
3,2711010300,동인동3가
4,2711010400,동인동4가
5,2711010500,삼덕동1가
6,2711010600,삼덕동2가
7,2711010700,삼덕동3가
8,2711010800,봉산동
9,2711010900,장관동
10,2711011000,상서동


In [13]:
dong_n_code = dong_code_df.values
# make dictionary
dong_n_code = dict(dong_n_code)

# switch key-value {dongName: dongCode}
dong_n_code = {y:x for x,y in dong_n_code.items()}

In [14]:
dong_n_code

{'계산동1가': 2711013800,
 '계산동2가': 2711013900,
 '공평동': 2711012200,
 '교동': 2711012500,
 '남산동': 2711015600,
 '남성로': 2711013700,
 '남일동': 2711011700,
 '달성동': 2711015500,
 '대봉동': 2711015700,
 '대신동': 2711015400,
 '대안동': 2711014700,
 '덕산동': 2711011200,
 '도원동': 2711012900,
 '동문동': 2711012000,
 '동산동': 2711014000,
 '동성로1가': 2711014800,
 '동성로2가': 2711012300,
 '동성로3가': 2711011900,
 '동인동1가': 2711010100,
 '동인동2가': 2711010200,
 '동인동3가': 2711010300,
 '동인동4가': 2711010400,
 '동일동': 2711011600,
 '문화동': 2711012100,
 '봉산동': 2711010800,
 '북내동': 2711015300,
 '북성로1가': 2711015000,
 '북성로2가': 2711014600,
 '사일동': 2711011500,
 '삼덕동1가': 2711010500,
 '삼덕동2가': 2711010600,
 '삼덕동3가': 2711010700,
 '상덕동': 2711012700,
 '상서동': 2711011000,
 '서내동': 2711014500,
 '서문로1가': 2711014400,
 '서문로2가': 2711014100,
 '서성로1가': 2711013400,
 '서성로2가': 2711014200,
 '서야동': 2711013300,
 '수동': 2711011100,
 '수창동': 2711013000,
 '시장북로': 2711013500,
 '완전동': 2711012800,
 '용덕동': 2711012600,
 '인교동': 2711013200,
 '장관동': 2711010900,
 '전동': 2711011800,
 '종로1가

In [15]:
# 기존 데이터에 우리가 방금 정제한 법정동코드 표를 적용합니다.
# 법정동 이름으로, 법정동: 코드 딕셔너리에서 값 가져와서 새로운 column 생성
df['법정동코드'] = df['법정동'].apply(lambda x: dong_n_code[x])

In [16]:
df.head()

,거래금액,건축년도,년,법정동,아파트,월,일,전용면적,지번,지역코드,층,법정동코드
0,14500,2007,2007,동인동1가,삼승슈퍼타워,8,1~10,83.50,362,27110,7,2711010100
1,15300,2007,2007,동인동1가,삼승슈퍼타워,8,21~31,83.50,362,27110,14,2711010100
2,16920,2007,2007,동인동1가,삼승슈퍼타워,8,21~31,83.50,362,27110,11,2711010100
3,16000,2006,2007,봉산동,봉산뜨란채,8,1~10,75.95,110,27110,3,2711010800
4,9200,1975,2007,수동,희도맨션,8,1~10,101.26,101,27110,8,2711011100


# 추가 데이터 ( 아파트 세부데이터) 얻기

## api를 이용해 아파트 세부 데이터를 얻으려면 아파트 코드가 필요합니다.

In [17]:
# 법정동 코드의 set값 얻기
loadCode = set(df['법정동코드'])
# 셋에서 리스트로 
loadCode = list(loadCode)

In [18]:
apt_code_dic = {}
for code in loadCode:
    # 법정동번호로 api 호출해서 법정동에 포함된 모든 아파트 목록 및 아파트 코드 받아오기
    key = '&ServiceKey=이것도 아까처럼 자신의 키를 받아서 복사붙여넣기하시면 됩니다'
    URL = 'http://apis.data.go.kr/1611000/AptListService/getLegaldongAptList?loadCode=' + str(code) + key
    res = urllib.request.urlopen(URL)
    # xml로 파싱
    soup = BeautifulSoup(res, "xml")
    # 모든 아이템 찾기
    items = soup.find_all("item")
    
    for item in items:
        apt_code_dic.update({item.find("kaptName").text: item.find("kaptCode").text})

# 혹시 몰라서 받은 파일을 텍스트 파일로 저장합니다.
with open('apt_code_dic.txt', 'wt') as file:
     json.dump(apt_code_dic, file)

In [19]:
# 파일읽기
with open('apt_code_dic.txt', 'r') as file:
    apt_code_dic = json.load(file)
    
    
apt_code_dic

{'극동스타클래스남산': 'A70044005',
 '까치아파트': 'A70075301',
 '남산그린타운': 'A70044004',
 '남산휴먼시아1단지': 'A70044002',
 '남산휴먼시아2단지': 'A70044003',
 '노마즈하우스': 'A70080001',
 '대봉청구타운': 'A70081001',
 '대봉태왕아너스아파트': 'A10027678',
 '대봉화성파크드림': 'A70081201',
 '대신센트럴자이': 'A10028088',
 '동산맨션': 'A70082101',
 '동인동삼정그린코아': 'A70042401',
 '동인시티타운': 'A70042101',
 '라이프아파트': 'A70011101',
 '반월당삼정그린코아(아파트)': 'A70044202',
 '보성송림맨션': 'A70072601',
 '보성황실타운': 'A70075102',
 '봉산뜨란채': 'A70082301',
 '삼덕청아람리슈빌': 'A70041301',
 '센트로팰리스': 'A70081003',
 '신성미소시티': 'A70071501',
 '인터불고코아시스': 'A70072401',
 '청운맨션': 'A70076102',
 '태왕아너스스카이': 'A70032001',
 '효성해링턴플레이스': 'A70080201'}

In [20]:
# 방금 읽어들인 딕셔너리를 이용해서
# 기존 데이터에 아파트코드 열을 새로 만들어 넣습니다.
df['아파트코드'] = df['아파트'].apply(lambda x: apt_code_dic[x] if x in apt_code_dic else None)

In [21]:
df

,거래금액,건축년도,년,법정동,아파트,월,일,전용면적,지번,지역코드,층,법정동코드,아파트코드
0,14500,2007,2007,동인동1가,삼승슈퍼타워,8,1~10,83.50,362,27110,7,2711010100,None
1,15300,2007,2007,동인동1가,삼승슈퍼타워,8,21~31,83.50,362,27110,14,2711010100,None
2,16920,2007,2007,동인동1가,삼승슈퍼타워,8,21~31,83.50,362,27110,11,2711010100,None
3,16000,2006,2007,봉산동,봉산뜨란채,8,1~10,75.95,110,27110,3,2711010800,A70082301
4,9200,1975,2007,수동,희도맨션,8,1~10,101.26,101,27110,8,2711011100,None
5,5800,1985,2007,태평로1가,태평라이프,8,1~10,54.60,1-187,27110,8,2711012400,None
6,5500,1985,2007,태평로1가,태평라이프,8,1~10,54.60,1-187,27110,6,2711012400,None
7,5100,1976,2007,도원동,도원,8,1~10,38.91,3-5,27110,2,2711012900,None
8,12500,2007,2007,동산동,석우리치타운,8,1~10,81.90,5-2,27110,4,2711014000,None
9,12500,2007,2007,동산동,석우리치타운,8,11~20,81.90,5-2,27110,3,2711014000,None


몇몇 아파트는 아파트코드가 없습니다.

### 문제가 있네요. 거래내역 데이터 (아파트)와  법정동 아파트목록 데이터 (아파트) 가 매칭이 되지 않습니다.
 수작업으로 매칭시킴
 None 값을 가진 13 개의 아파트를 수작업으로 매칭

In [22]:
missMatch = set(df['아파트'].loc[df['아파트코드'].isnull()])
missMatch

{'건영맨션',
 '경진테크빌(101동)',
 '경진테크빌(102동)',
 '광명',
 '극동(고려)',
 '금오맨션',
 '까치',
 '나누리',
 '남문시장2지구',
 '남산모던',
 '대봉 태왕아너스',
 '대봉맨션',
 '대봉화성그린빌',
 '대신',
 '대신 센트럴자이',
 '대흥아트',
 '도원',
 '동남',
 '동삼(88-0)',
 '동서',
 '동인(279-1)',
 '동인(341-2)',
 '동인태평78',
 '미소시티',
 '반월당삼정그린코아',
 '반월당효성해링턴플레이스',
 '부림베스트빌',
 '삼덕드림타운',
 '삼덕아이파크',
 '삼덕인베스트',
 '삼덕청아람',
 '삼성솔라',
 '삼승슈퍼타워',
 '삼익맨션',
 '서성로훼밀리타운',
 '석우리치타운',
 '센텀리움',
 '송림맨션',
 '시티타운',
 '심미한마음',
 '아진맨션',
 '아트빌',
 '지성하이츠',
 '청구맨션',
 '태평',
 '태평라이프',
 '한양가든테라스',
 '허브스카이',
 '현서그린',
 '홍길',
 '황궁',
 '희도맨션'}

잘못된 아파트이름 다 변경

In [24]:
# 아파트값 변경을위해 딕셔너리 생성 {old_apt_name: new_apt_name}
old_new = {'극동(고려)': '극동스타클래스남산', '까치': '까치아파트', '대봉 태왕아너스': '대봉태왕아너스아파트', '대신 센트럴자이': '대신센트럴자이',
          '미소시티': '신성미소시티', '반월당삼정그린코아': '반월당삼정그린코아(아파트)', '반월당효성해링턴플레이스': '효성해링턴플레이스',
           '삼덕청아람': '삼덕청아람리슈빌', '송림맨션': '보성송림맨션', '시티타운': '동인시티타운', '청구맨션': '대봉청구타운', '태평라이프': '라이프아파트'}
old_new

{'극동(고려)': '극동스타클래스남산',
 '까치': '까치아파트',
 '대봉 태왕아너스': '대봉태왕아너스아파트',
 '대신 센트럴자이': '대신센트럴자이',
 '미소시티': '신성미소시티',
 '반월당삼정그린코아': '반월당삼정그린코아(아파트)',
 '반월당효성해링턴플레이스': '효성해링턴플레이스',
 '삼덕청아람': '삼덕청아람리슈빌',
 '송림맨션': '보성송림맨션',
 '시티타운': '동인시티타운',
 '청구맨션': '대봉청구타운',
 '태평라이프': '라이프아파트'}

### groupby: 나누고 -> 적용하고 -> 합치기


- groupby로 데이터프레임의 아파트 칼럼에서 까치인 sub데이터프레임으로 나누고, 까치를 까치아파트로 다 바꾸고 다시 원래 데이터프레임에 합친다.
- group = df.groupby(df['아파트']=='까치').apply(lambda x: x.replace('까치', '까치아파트'))
- df[df['아파트']=='까치'] = group

In [25]:
# 안맞는 아파트 이름을 다 변경합니다.
for old, new in old_new.items():
    group = df.groupby(df['아파트']==old).apply(lambda x: x.replace(old, new))
    df[df['아파트']==old] = group

In [26]:
# 그리고 아파트 코드를 할당해줍니다.
df['아파트코드'] = df['아파트'].apply(lambda x: apt_code_dic[x] if x in apt_code_dic else None)

In [27]:
df

,거래금액,건축년도,년,법정동,아파트,월,일,전용면적,지번,지역코드,층,법정동코드,아파트코드
0,14500,2007,2007,동인동1가,삼승슈퍼타워,8,1~10,83.50,362,27110,7,2711010100,None
1,15300,2007,2007,동인동1가,삼승슈퍼타워,8,21~31,83.50,362,27110,14,2711010100,None
2,16920,2007,2007,동인동1가,삼승슈퍼타워,8,21~31,83.50,362,27110,11,2711010100,None
3,16000,2006,2007,봉산동,봉산뜨란채,8,1~10,75.95,110,27110,3,2711010800,A70082301
4,9200,1975,2007,수동,희도맨션,8,1~10,101.26,101,27110,8,2711011100,None
5,5800,1985,2007,태평로1가,라이프아파트,8,1~10,54.60,1-187,27110,8,2711012400,A70011101
6,5500,1985,2007,태평로1가,라이프아파트,8,1~10,54.60,1-187,27110,6,2711012400,A70011101
7,5100,1976,2007,도원동,도원,8,1~10,38.91,3-5,27110,2,2711012900,None
8,12500,2007,2007,동산동,석우리치타운,8,1~10,81.90,5-2,27110,4,2711014000,None
9,12500,2007,2007,동산동,석우리치타운,8,11~20,81.90,5-2,27110,3,2711014000,None


In [28]:
# 아파트코드가 있는지 없는지 확인
set(df['아파트'].loc[df['아파트코드'].isnull()])

{'건영맨션',
 '경진테크빌(101동)',
 '경진테크빌(102동)',
 '광명',
 '금오맨션',
 '나누리',
 '남문시장2지구',
 '남산모던',
 '대봉맨션',
 '대봉화성그린빌',
 '대신',
 '대흥아트',
 '도원',
 '동남',
 '동삼(88-0)',
 '동서',
 '동인(279-1)',
 '동인(341-2)',
 '동인태평78',
 '부림베스트빌',
 '삼덕드림타운',
 '삼덕아이파크',
 '삼덕인베스트',
 '삼성솔라',
 '삼승슈퍼타워',
 '삼익맨션',
 '서성로훼밀리타운',
 '석우리치타운',
 '센텀리움',
 '심미한마음',
 '아진맨션',
 '아트빌',
 '지성하이츠',
 '태평',
 '한양가든테라스',
 '허브스카이',
 '현서그린',
 '홍길',
 '황궁',
 '희도맨션'}

40개의 아파트가 아파트코드가 없습니다.

# Get Extra Data

K-APT api사용
1. 아파트 코드 사용해서
2. 아파트 단지정보 받아오기


In [29]:
# 아파트 디테일 정보 가져오기

# 필요한 데이터 키값
list_of_data = ['CODE_HALL', 'CODE_HEAT', 'CODE_SEC', 'CODE_STR', 'CONVENIENT_FACILITY', 'EDUCATION_FACILITY',
                'KAPTD_PCNT','KAPTD_PCNTU', 'KAPTD_WTIMEBUS', 'KAPTD_WTIMESUB', 'KAPT_DONG_CNT', 'KAPT_MGR_CNT',
                'KAPTD_ECNTP', 'KAPT_USEDATE', 'SUBWAY_LINE', 'SUBWAY_STATION', 'WELFARE_FACILITY']

# 필요한 모든 아파트 세부정보
details_data = {}

for v in apt_code_dic.values():
    response = urllib.request.urlopen("http://www.k-apt.go.kr/kaptinfo/getKaptInfo_detail.do?kapt_code=" + str(v))
    data = json.loads(response.read().decode('utf-8'))
    apt_data = {}
    for tem in list_of_data:
        apt_data.update({tem: data['resultMap_kapt'][tem]})
    details_data.update({v: apt_data})

In [30]:
'''
아파트 세부정보 데이터의 구조
{
    {아파트코드1: {  데이터1: 값1, 
                        데이터2: 값2,
                        데이터3: 값3,
                        ...  : ...
                      }
    },
    {아파트코드2: {  데이터1: 값1, 
                        데이터2: 값2,
                        데이터3: 값3,
                        ...  : ...
                      }
    },
}
'''

details_data

{'A10027678': {'CODE_HALL': '계단식',
  'CODE_HEAT': '개별난방',
  'CODE_SEC': '위탁관리',
  'CODE_STR': '철근콘크리트구조',
  'CONVENIENT_FACILITY': '-',
  'EDUCATION_FACILITY': '-',
  'KAPTD_ECNTP': 10,
  'KAPTD_PCNT': '102',
  'KAPTD_PCNTU': '400',
  'KAPTD_WTIMEBUS': '5분이내',
  'KAPTD_WTIMESUB': '5~10분이내',
  'KAPT_DONG_CNT': '6',
  'KAPT_MGR_CNT': '5',
  'KAPT_USEDATE': '2015-11-25',
  'SUBWAY_LINE': '2호선',
  'SUBWAY_STATION': '대구2호선',
  'WELFARE_FACILITY': '관리사무소, 노인정, 보육시설, 문고, 주민공동시설, 어린이놀이터, 자전거보관소'},
 'A10028088': {'CODE_HALL': '계단식',
  'CODE_HEAT': '개별난방',
  'CODE_SEC': '위탁관리',
  'CODE_STR': '철근콘크리트구조',
  'CONVENIENT_FACILITY': '관공서(대구중부소방서) 병원(계명대학교 동산병원) 백화점(현대백화점,동아백화점) 대형상가(서문시장) 공원(달성공원) 기타()',
  'EDUCATION_FACILITY': '초등학교(계성초교,대구남산초교) 중학교(계성중교) 고등학교(계성고교) 대학교(대구카톨릭대학교)',
  'KAPTD_ECNTP': 24,
  'KAPTD_PCNT': '175',
  'KAPTD_PCNTU': '1321',
  'KAPTD_WTIMEBUS': '5~10분이내',
  'KAPTD_WTIMESUB': '5~10분이내',
  'KAPT_DONG_CNT': '13',
  'KAPT_MGR_CNT': '8',
  'KAPT_USEDATE': '2015-04-15',
  'SUBWAY_

받은 데이터를 csv로 저장합니다. 혹시모르니까요 ^~^

In [31]:
# 딕셔너리를 csv파일로 저장합니다.
with open('apt_details.csv', "w", newline='') as csv_file:
    writer = csv.writer(csv_file, delimiter=',')
    # 헤더를 쓰고
    writer.writerow(list_of_data)
    
    # 모든키를 순회
    for k, v in details_data.items():
        # 리스트 생성
        contents = []
        # 키를 먼저 넣고
        contents.append(k)  # key
        
        # 모든값을 순회
        for i in range(len(list_of_data)):
            contents.append(details_data[k][list_of_data[i]])  # 값들
        writer.writerow(contents)
            

In [32]:
# 저장된 csv파일을 다시 result로 불러들입니다.
result = pd.read_csv('apt_details.csv')
result

,CODE_HALL,CODE_HEAT,CODE_SEC,CODE_STR,CONVENIENT_FACILITY,EDUCATION_FACILITY,KAPTD_PCNT,KAPTD_PCNTU,KAPTD_WTIMEBUS,KAPTD_WTIMESUB,KAPT_DONG_CNT,KAPT_MGR_CNT,KAPTD_ECNTP,KAPT_USEDATE,SUBWAY_LINE,SUBWAY_STATION,WELFARE_FACILITY
A70071501,계단식,개별난방,위탁관리,철골철근콘크리트구조,"관공서(중앙파출소, 성내2동 주민센터, 대구시 동부 교육청, 남산지구대) 병원(동산...","초등학교(남산초,) 중학교(성명여중) 고등학교(신명고) 대학교(계명대 의과대학)",7,605,5분이내,5분이내,2,5,5,2007-10-09,"1호선, 2호선",반월당,"관리사무소, 노인정, 주민공동시설, 어린이놀이터, 자전거보관소"
A70041301,계단식,개별난방,위탁관리,철근콘크리트구조,"관공서(중구청,동인동사무소,삼덕동우체국,동인3가파출소,국민연금관리공단남대구지사) 병...","초등학교(사대부초,동덕초,동인초,삼덕초) 중학교(대구중앙중,신명여중) 고등학교(남산...",8,930,5분이내,5분이내,7,6,14,2013-04-03,2호선,경대병원,"관리사무소, 노인정, 보육시설, 문고, 주민공동시설, 어린이놀이터, 자전거보관소"
A70076102,계단식,중앙난방,위탁관리,철근콘크리트구조,"관공서(국민연금관리공단남대구지사,대구광역시교육위원회,대봉동사무소,대봉우체국,대봉치안...","초등학교(동덕초,삼덕초,대봉초) 중학교(신명여중) 고등학교(남산고) 대학교(경대치대)",713,0,5분이내,10~15분이내,7,8,27,1986-12-05,2호선,경대병원,"관리사무소, 노인정, 어린이놀이터, 자전거보관소"
A70081003,계단식,개별난방,위탁관리,철근콘크리트구조,"관공서(대봉도서관,대구동부교육청,시립중앙도서관) 병원(경대치과병원) 백화점(대구백화...","초등학교(대구초,경대사대부속초,삼덕초) 중학교(경대사대부속중,제일여중,신명여중) 고...",0,1270,5분이내,5분이내,7,14,16,2007-03-23,2호선,경대병원역,"관리사무소, 노인정, 보육시설, 문고, 주민공동시설, 어린이놀이터, 휴게시설, 커뮤..."
A10027678,계단식,개별난방,위탁관리,철근콘크리트구조,-,-,102,400,5분이내,5~10분이내,6,5,10,2015-11-25,2호선,대구2호선,"관리사무소, 노인정, 보육시설, 문고, 주민공동시설, 어린이놀이터, 자전거보관소"
A70081001,복도식,개별난방,자치관리,철근콘크리트구조,"관공서(대봉동사무소,대봉우체국,대봉치안센터) 병원(인제병원,경대치과병원) 백화점(대...","초등학교(영선초,삼덕초,동인초) 중학교(경북대사범부속중,신명여중,제일중) 고등학교(...",87,0,5분이내,5분이내,2,1,4,1978-11-18,2호선,경대병원,"관리사무소, 어린이놀이터, 자전거보관소"
A70081201,계단식,개별난방,위탁관리,철근콘크리트구조,"관공서(대봉동사무소,대봉우체국,대봉치안센터) 병원(남부보건소) 백화점(대백프라자) ...","초등학교(대구초,경북대사범부속초,명덕초,영선초) 중학교(신명여중,제일중,경북대사범부...",123,181,5~10분이내,5분이내,3,3,11,2006-05-22,3호선,명덕,"관리사무소, 노인정, 어린이놀이터, 자전거보관소"
A70080001,복도식,개별난방,자치관리,철근콘크리트구조,"관공서(교동119안전센터,대구역전우체국, 대구시청,중구청) 병원(경북대학교 병원) ...",대학교(경북대학교 의대),0,218,10~15분이내,5~10분이내,1,4,0,2014-01-21,1호선,-,관리사무소
A70011101,복도식,개별난방,자치관리,철근콘크리트구조,"관공서(대구광역시청,대구세무서,중부경찰서,대구우체국,대구지방병무청) 병원(곽병원) ...","초등학교(옥산초,종로초) 중학교(경명여중) 고등학교(경명여고)",80,76,5분이내,5~10분이내,1,2,2,1985-07-15,1호선,대구,"관리사무소, 노인정, 자전거보관소"
A70044004,혼합식,개별난방,위탁관리,철근콘크리트구조,"관공서(대구도시가스,동사무소,계명대학우체국,대구도시가스,대구중부소방서) 병원(동산의...","초등학교(계성초,남산초,내당초) 중학교(구남중,계성중,경구중,신명여중) 고등학교(구...",67,798,5분이내,5~10분이내,8,6,0,2005-04-27,2호선,반고개,"관리사무소, 노인정, 보육시설, 문고, 주민공동시설, 어린이놀이터, 자전거보관소"


result 표는 아파트코드가 인덱스로 되어있습니다. 고쳐봅시다.

In [33]:
result.reset_index(inplace=True)
result.rename(columns={'index': 'APT_CODE'}, inplace=True)
result

,APT_CODE,CODE_HALL,CODE_HEAT,CODE_SEC,CODE_STR,CONVENIENT_FACILITY,EDUCATION_FACILITY,KAPTD_PCNT,KAPTD_PCNTU,KAPTD_WTIMEBUS,KAPTD_WTIMESUB,KAPT_DONG_CNT,KAPT_MGR_CNT,KAPTD_ECNTP,KAPT_USEDATE,SUBWAY_LINE,SUBWAY_STATION,WELFARE_FACILITY
0,A70071501,계단식,개별난방,위탁관리,철골철근콘크리트구조,"관공서(중앙파출소, 성내2동 주민센터, 대구시 동부 교육청, 남산지구대) 병원(동산...","초등학교(남산초,) 중학교(성명여중) 고등학교(신명고) 대학교(계명대 의과대학)",7,605,5분이내,5분이내,2,5,5,2007-10-09,"1호선, 2호선",반월당,"관리사무소, 노인정, 주민공동시설, 어린이놀이터, 자전거보관소"
1,A70041301,계단식,개별난방,위탁관리,철근콘크리트구조,"관공서(중구청,동인동사무소,삼덕동우체국,동인3가파출소,국민연금관리공단남대구지사) 병...","초등학교(사대부초,동덕초,동인초,삼덕초) 중학교(대구중앙중,신명여중) 고등학교(남산...",8,930,5분이내,5분이내,7,6,14,2013-04-03,2호선,경대병원,"관리사무소, 노인정, 보육시설, 문고, 주민공동시설, 어린이놀이터, 자전거보관소"
2,A70076102,계단식,중앙난방,위탁관리,철근콘크리트구조,"관공서(국민연금관리공단남대구지사,대구광역시교육위원회,대봉동사무소,대봉우체국,대봉치안...","초등학교(동덕초,삼덕초,대봉초) 중학교(신명여중) 고등학교(남산고) 대학교(경대치대)",713,0,5분이내,10~15분이내,7,8,27,1986-12-05,2호선,경대병원,"관리사무소, 노인정, 어린이놀이터, 자전거보관소"
3,A70081003,계단식,개별난방,위탁관리,철근콘크리트구조,"관공서(대봉도서관,대구동부교육청,시립중앙도서관) 병원(경대치과병원) 백화점(대구백화...","초등학교(대구초,경대사대부속초,삼덕초) 중학교(경대사대부속중,제일여중,신명여중) 고...",0,1270,5분이내,5분이내,7,14,16,2007-03-23,2호선,경대병원역,"관리사무소, 노인정, 보육시설, 문고, 주민공동시설, 어린이놀이터, 휴게시설, 커뮤..."
4,A10027678,계단식,개별난방,위탁관리,철근콘크리트구조,-,-,102,400,5분이내,5~10분이내,6,5,10,2015-11-25,2호선,대구2호선,"관리사무소, 노인정, 보육시설, 문고, 주민공동시설, 어린이놀이터, 자전거보관소"
5,A70081001,복도식,개별난방,자치관리,철근콘크리트구조,"관공서(대봉동사무소,대봉우체국,대봉치안센터) 병원(인제병원,경대치과병원) 백화점(대...","초등학교(영선초,삼덕초,동인초) 중학교(경북대사범부속중,신명여중,제일중) 고등학교(...",87,0,5분이내,5분이내,2,1,4,1978-11-18,2호선,경대병원,"관리사무소, 어린이놀이터, 자전거보관소"
6,A70081201,계단식,개별난방,위탁관리,철근콘크리트구조,"관공서(대봉동사무소,대봉우체국,대봉치안센터) 병원(남부보건소) 백화점(대백프라자) ...","초등학교(대구초,경북대사범부속초,명덕초,영선초) 중학교(신명여중,제일중,경북대사범부...",123,181,5~10분이내,5분이내,3,3,11,2006-05-22,3호선,명덕,"관리사무소, 노인정, 어린이놀이터, 자전거보관소"
7,A70080001,복도식,개별난방,자치관리,철근콘크리트구조,"관공서(교동119안전센터,대구역전우체국, 대구시청,중구청) 병원(경북대학교 병원) ...",대학교(경북대학교 의대),0,218,10~15분이내,5~10분이내,1,4,0,2014-01-21,1호선,-,관리사무소
8,A70011101,복도식,개별난방,자치관리,철근콘크리트구조,"관공서(대구광역시청,대구세무서,중부경찰서,대구우체국,대구지방병무청) 병원(곽병원) ...","초등학교(옥산초,종로초) 중학교(경명여중) 고등학교(경명여고)",80,76,5분이내,5~10분이내,1,2,2,1985-07-15,1호선,대구,"관리사무소, 노인정, 자전거보관소"
9,A70044004,혼합식,개별난방,위탁관리,철근콘크리트구조,"관공서(대구도시가스,동사무소,계명대학우체국,대구도시가스,대구중부소방서) 병원(동산의...","초등학교(계성초,남산초,내당초) 중학교(구남중,계성중,경구중,신명여중) 고등학교(구...",67,798,5분이내,5~10분이내,8,6,0,2005-04-27,2호선,반고개,"관리사무소, 노인정, 보육시설, 문고, 주민공동시설, 어린이놀이터, 자전거보관소"


In [34]:
result.shape

(25, 18)


FEATURES:
-   CODE_HALL: 복도형식,
-   CODE_HEAT: 난방방식,
-   CODE_SEC: 관리방식,
-   CODE_STR: 건축구조,
-   CONVENIENT_FACILITY: 편의시설,
-   EDUCATION_FACILITY: 교육시설,
-   KAPTD_PCNT: 주차공간(지상),
-   KAPTD_PCNTU: 주차공간(지하),
-   KAPTD_WTIMEBUS: 버스정류장,
-   KAPTD_WTIMESUB: 지하철,
-   KAPT_DONG_CNT: 동 갯수,
-   KAPT_MGR_CNT: 일반관리인원,
-   KAPTD_ECNTP: 엘리베이터갯수,
-   KAPT_USEDATE: 사용승인일,
-   SUBWAY_LINE: 지하철노선,
-   SUBWAY_STATION: 지하철역,
-   WELFARE_FACILITY: 부대/복리 시설


In [35]:
# 기존의 데이터와 result(세부정보데이터)를 합치려면 공통의 primary key가 필요합니다.
# primary key를 아파트 코드로 정합니다.
result.rename(columns={'APT_CODE': '아파트코드'}, inplace=True)
result

,아파트코드,CODE_HALL,CODE_HEAT,CODE_SEC,CODE_STR,CONVENIENT_FACILITY,EDUCATION_FACILITY,KAPTD_PCNT,KAPTD_PCNTU,KAPTD_WTIMEBUS,KAPTD_WTIMESUB,KAPT_DONG_CNT,KAPT_MGR_CNT,KAPTD_ECNTP,KAPT_USEDATE,SUBWAY_LINE,SUBWAY_STATION,WELFARE_FACILITY
0,A70071501,계단식,개별난방,위탁관리,철골철근콘크리트구조,"관공서(중앙파출소, 성내2동 주민센터, 대구시 동부 교육청, 남산지구대) 병원(동산...","초등학교(남산초,) 중학교(성명여중) 고등학교(신명고) 대학교(계명대 의과대학)",7,605,5분이내,5분이내,2,5,5,2007-10-09,"1호선, 2호선",반월당,"관리사무소, 노인정, 주민공동시설, 어린이놀이터, 자전거보관소"
1,A70041301,계단식,개별난방,위탁관리,철근콘크리트구조,"관공서(중구청,동인동사무소,삼덕동우체국,동인3가파출소,국민연금관리공단남대구지사) 병...","초등학교(사대부초,동덕초,동인초,삼덕초) 중학교(대구중앙중,신명여중) 고등학교(남산...",8,930,5분이내,5분이내,7,6,14,2013-04-03,2호선,경대병원,"관리사무소, 노인정, 보육시설, 문고, 주민공동시설, 어린이놀이터, 자전거보관소"
2,A70076102,계단식,중앙난방,위탁관리,철근콘크리트구조,"관공서(국민연금관리공단남대구지사,대구광역시교육위원회,대봉동사무소,대봉우체국,대봉치안...","초등학교(동덕초,삼덕초,대봉초) 중학교(신명여중) 고등학교(남산고) 대학교(경대치대)",713,0,5분이내,10~15분이내,7,8,27,1986-12-05,2호선,경대병원,"관리사무소, 노인정, 어린이놀이터, 자전거보관소"
3,A70081003,계단식,개별난방,위탁관리,철근콘크리트구조,"관공서(대봉도서관,대구동부교육청,시립중앙도서관) 병원(경대치과병원) 백화점(대구백화...","초등학교(대구초,경대사대부속초,삼덕초) 중학교(경대사대부속중,제일여중,신명여중) 고...",0,1270,5분이내,5분이내,7,14,16,2007-03-23,2호선,경대병원역,"관리사무소, 노인정, 보육시설, 문고, 주민공동시설, 어린이놀이터, 휴게시설, 커뮤..."
4,A10027678,계단식,개별난방,위탁관리,철근콘크리트구조,-,-,102,400,5분이내,5~10분이내,6,5,10,2015-11-25,2호선,대구2호선,"관리사무소, 노인정, 보육시설, 문고, 주민공동시설, 어린이놀이터, 자전거보관소"
5,A70081001,복도식,개별난방,자치관리,철근콘크리트구조,"관공서(대봉동사무소,대봉우체국,대봉치안센터) 병원(인제병원,경대치과병원) 백화점(대...","초등학교(영선초,삼덕초,동인초) 중학교(경북대사범부속중,신명여중,제일중) 고등학교(...",87,0,5분이내,5분이내,2,1,4,1978-11-18,2호선,경대병원,"관리사무소, 어린이놀이터, 자전거보관소"
6,A70081201,계단식,개별난방,위탁관리,철근콘크리트구조,"관공서(대봉동사무소,대봉우체국,대봉치안센터) 병원(남부보건소) 백화점(대백프라자) ...","초등학교(대구초,경북대사범부속초,명덕초,영선초) 중학교(신명여중,제일중,경북대사범부...",123,181,5~10분이내,5분이내,3,3,11,2006-05-22,3호선,명덕,"관리사무소, 노인정, 어린이놀이터, 자전거보관소"
7,A70080001,복도식,개별난방,자치관리,철근콘크리트구조,"관공서(교동119안전센터,대구역전우체국, 대구시청,중구청) 병원(경북대학교 병원) ...",대학교(경북대학교 의대),0,218,10~15분이내,5~10분이내,1,4,0,2014-01-21,1호선,-,관리사무소
8,A70011101,복도식,개별난방,자치관리,철근콘크리트구조,"관공서(대구광역시청,대구세무서,중부경찰서,대구우체국,대구지방병무청) 병원(곽병원) ...","초등학교(옥산초,종로초) 중학교(경명여중) 고등학교(경명여고)",80,76,5분이내,5~10분이내,1,2,2,1985-07-15,1호선,대구,"관리사무소, 노인정, 자전거보관소"
9,A70044004,혼합식,개별난방,위탁관리,철근콘크리트구조,"관공서(대구도시가스,동사무소,계명대학우체국,대구도시가스,대구중부소방서) 병원(동산의...","초등학교(계성초,남산초,내당초) 중학교(구남중,계성중,경구중,신명여중) 고등학교(구...",67,798,5분이내,5~10분이내,8,6,0,2005-04-27,2호선,반고개,"관리사무소, 노인정, 보육시설, 문고, 주민공동시설, 어린이놀이터, 자전거보관소"


In [36]:
# left merge
# 표 merge에 대해 잘 설명된 곳 참조하세요: 'https://chrisalbon.com/python/pandas_join_merge_dataframe.html'
final_df = pd.merge(df, result, how='left', on='아파트코드')
final_df

,거래금액,건축년도,년,법정동,아파트,월,일,전용면적,지번,지역코드,...,KAPTD_PCNTU,KAPTD_WTIMEBUS,KAPTD_WTIMESUB,KAPT_DONG_CNT,KAPT_MGR_CNT,KAPTD_ECNTP,KAPT_USEDATE,SUBWAY_LINE,SUBWAY_STATION,WELFARE_FACILITY
0,14500,2007,2007,동인동1가,삼승슈퍼타워,8,1~10,83.50,362,27110,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,15300,2007,2007,동인동1가,삼승슈퍼타워,8,21~31,83.50,362,27110,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,16920,2007,2007,동인동1가,삼승슈퍼타워,8,21~31,83.50,362,27110,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,16000,2006,2007,봉산동,봉산뜨란채,8,1~10,75.95,110,27110,...,184.0,5~10분이내,10~15분이내,3.0,3.0,0.0,2006-08-18,2호선,경대병원,"관리사무소, 노인정, 주민공동시설, 어린이놀이터, 자전거보관소"
4,9200,1975,2007,수동,희도맨션,8,1~10,101.26,101,27110,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5800,1985,2007,태평로1가,라이프아파트,8,1~10,54.60,1-187,27110,...,76.0,5분이내,5~10분이내,1.0,2.0,2.0,1985-07-15,1호선,대구,"관리사무소, 노인정, 자전거보관소"
6,5500,1985,2007,태평로1가,라이프아파트,8,1~10,54.60,1-187,27110,...,76.0,5분이내,5~10분이내,1.0,2.0,2.0,1985-07-15,1호선,대구,"관리사무소, 노인정, 자전거보관소"
7,5100,1976,2007,도원동,도원,8,1~10,38.91,3-5,27110,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,12500,2007,2007,동산동,석우리치타운,8,1~10,81.90,5-2,27110,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,12500,2007,2007,동산동,석우리치타운,8,11~20,81.90,5-2,27110,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
# 열의 이름을 한국어로 바꿔줍시다
final_df.rename(columns={'년': '거래년도',
'월': '거래월',
'일': '거래일',
'CODE_HALL': '복도형식',
'CODE_HEAT': '난방방식',
'CODE_SEC': '관리방식',
'CODE_STR': '건축구조',
'CONVENIENT_FACILITY': '편의시설',
'EDUCATION_FACILITY': '교육시설',
'KAPTD_PCNT': '주차공간(지상)',
'KAPTD_PCNTU': '주차공간(지하)',
'KAPTD_WTIMEBUS': '버스정류장',
'KAPTD_WTIMESUB': '지하철',
'KAPT_DONG_CNT': '동갯수',
'KAPT_MGR_CNT': '일반관리인원',
'KAPTD_ECNTP': '엘리베이터갯수',
'KAPT_USEDATE': '사용승인일',
'SUBWAY_LINE': '지하철노선',
'SUBWAY_STATION': '지하철역',
'WELFARE_FACILITY': '부대/복리시설'}, inplace=True)

In [38]:
# 합쳐진 최종 데이터를 엑셀로 저장합니다.
writer = pd.ExcelWriter('data.xlsx')
final_df.to_excel(writer, 'Sheet1', index=True)
writer.save()

# 데이터 모으기 끝!!

In [39]:
final_df['아파트코드'].isnull().sum()
1035 / 6926 * 100

14.943690441813457

전체 데이터 : 6926개
세부정보가 없는 데이터 : 1035개 (15%)

# 참조: 
데이터 모으기
- http://goodvc.tistory.com/10